In [1]:
import boto3

In [2]:
region_name = 'eu-west-1'
roleArn = 'arn:aws:iam::729524366783:role/TestBedrockAgentRole'

bedrock_agent = boto3.client(service_name='bedrock-agent', region_name=region_name)

In [3]:
create_agent_response = bedrock_agent.create_agent(
    agentName='mugs-customer-support-agent',
    foundationModel='anthropic.claude-3-haiku-20240307-v1:0',
    instruction="""You are an advanced AI agent acting as a front line customer support agent.""",
    agentResourceRoleArn=roleArn
)
create_agent_response

{'ResponseMetadata': {'RequestId': '0b810e4e-8ad0-4a78-a5de-9a66ac8f3a0d',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sun, 09 Mar 2025 17:41:39 GMT',
   'content-type': 'application/json',
   'content-length': '580',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0b810e4e-8ad0-4a78-a5de-9a66ac8f3a0d',
   'x-amz-apigw-id': 'HK3hAFQDDoEEHUQ=',
   'x-amzn-trace-id': 'Root=1-67cdd2d2-1d0208a3242490031527b1e6'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:eu-west-1:729524366783:agent/WWGIQFRBTA',
  'agentCollaboration': 'DISABLED',
  'agentId': 'WWGIQFRBTA',
  'agentName': 'mugs-customer-support-agent',
  'agentResourceRoleArn': 'arn:aws:iam::729524366783:role/TestBedrockAgentRole',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2025, 3, 9, 17, 41, 38, 867019, tzinfo=tzutc()),
  'foundationModel': 'anthropic.claude-3-haiku-20240307-v1:0',
  'idleSessionTTLInSeconds': 600,
  'instruction': 'You are an advanced AI agent acting as a front line

In [4]:
agentId = create_agent_response['agent']['agentId']
agentId

'WWGIQFRBTA'

In [5]:
from helper import *

In [6]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='NOT_PREPARED'
)

Waiting for agent status of 'NOT_PREPARED'...
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.


In [7]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

{'ResponseMetadata': {'RequestId': 'b1df8f49-53d2-4faf-9865-70d0ff187ad4',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sun, 09 Mar 2025 17:41:46 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b1df8f49-53d2-4faf-9865-70d0ff187ad4',
   'x-amz-apigw-id': 'HK3iJFa_DoEEMUA=',
   'x-amzn-trace-id': 'Root=1-67cdd2da-28e6bc191f91c3a347c13528'},
  'RetryAttempts': 0},
 'agentId': 'WWGIQFRBTA',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 3, 9, 17, 41, 46, 377242, tzinfo=tzutc())}

In [8]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [9]:
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agentId,
    agentAliasName='MyAgentAlias',
)

agentAliasId = create_agent_alias_response['agentAlias']['agentAliasId']

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


In [10]:
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name=region_name)

In [11]:
import uuid

message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

invoke_agent_response = bedrock_agent_runtime.invoke_agent(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    endSession=False,
    enableTrace=True,
)

In [12]:
event_stream = invoke_agent_response["completion"]

for event in event_stream:
    print(event)

{'trace': {'agentAliasId': 'YJQCHT7ZRL', 'agentId': 'WWGIQFRBTA', 'agentVersion': '1', 'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:eu-west-1:729524366783:agent-alias/WWGIQFRBTA/YJQCHT7ZRL'}], 'eventTime': datetime.datetime(2025, 3, 9, 17, 44, 8, 835927, tzinfo=tzutc()), 'sessionId': '1dbb2150-60b8-4942-84dc-484ea2d1e554', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</invoke>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":" You are an advanced AI agent acting as a front line customer support agent. You have been provided with a set of functions to answer the user\'s question. You must call the functions in the format below: <function_calls>   <invoke>     <tool_name>$TOOL_NAME</tool_name>     <parameters>       <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>       ...     </parameters>   </invoke> </function_calls> Here are the functions available:

EventStreamError: An error occurred (accessDeniedException) when calling the InvokeAgent operation: Access denied when calling Bedrock. Check your request permissions and retry the request.

In [15]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

In [16]:
invoke_agent_and_print(
    agentAliasId=agentAliasId,
    agentId=agentId,
    sessionId=sessionId,
    inputText=message,
    enableTrace=True,
)

User: Hello, I bought a mug from your store yesterday, and it broke. I want
to return it.

Agent: 

EventStreamError: An error occurred (accessDeniedException) when calling the InvokeAgent operation: Access denied when calling Bedrock. Check your request permissions and retry the request.